 #  1.IMPORT THƯ VIỆN

In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import os
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


# 2.CHUẨN BỊ DỮ LIỆU

In [2]:
INPUT_SIZE = 224  # Kích thước ảnh mà ResNet50 yêu cầu

# Đường dẫn tới các thư mục ảnh
image_directory = 'datasets/'

# Lấy danh sách các ảnh từ các thư mục
no_tumor_images = os.listdir(image_directory + 'crop_no/')
yes_tumor_images = os.listdir(image_directory + 'crop_yes/')

dataset = []
labels = []

# Đọc và xử lý ảnh không có khối u
for image_name in no_tumor_images:
    if image_name.endswith('.jpg'):
        image_path = os.path.join(image_directory + 'crop_no/', image_name)
        image = cv2.imread(image_path)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        labels.append(0)

# Đọc và xử lý ảnh có khối u
for image_name in yes_tumor_images:
    if image_name.endswith('.jpg'):
        image_path = os.path.join(image_directory + 'crop_yes/', image_name)
        image = cv2.imread(image_path)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        labels.append(1)

# Chuyển đổi dữ liệu thành numpy arrays
dataset = np.array(dataset)
labels = np.array(labels)

In [3]:
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu: Đưa giá trị pixel về khoảng từ 0 đến 1
x_train_normalized = x_train / 255.0
x_test_normalized = x_test / 255.0


# One-hot encode nhãn (labels)
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)


# 3.TẢI MÔ HÌNH RESNET50 PRE-TRAINED

In [4]:
# Load ResNet50 với trọng số pre-trained, bỏ qua phần fully-connected layer cuối cùng
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))

# Đóng băng các lớp của ResNet50 để không huấn luyện lại
for layer in base_model.layers:
    layer.trainable = False


# 4. XÂY DỰNG MÔ HÌNH

In [5]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='sigmoid')  
])

model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
 

# 5.HUẤN LUYỆN MÔ HÌNH

In [6]:
model.fit(
    x_train_normalized,
    y_train,
    batch_size=32,  
    epochs=10,
    validation_data=(x_test_normalized, y_test),
    callbacks=[reduce_lr, early_stopping],
    shuffle=True
)


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 61s 865ms/step - accuracy: 0.5400 - loss: 0.9301 - val_accuracy: 0.6120 - val_loss: 0.6529 - learning_rate: 0.0010
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 53s 844ms/step - accuracy: 0.6022 - loss: 0.6787 - val_accuracy: 0.6220 - val_loss: 0.6478 - learning_rate: 0.0010
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 52s 823ms/step - accuracy: 0.5997 - loss: 0.6796 - val_accuracy: 0.6260 - val_loss: 0.6452 - learning_rate: 0.0010
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 52s 830ms/step - accuracy: 0.6231 - loss: 0.6435 - val_accuracy: 0.6320 - val_loss: 0.6418 - learning_rate: 0.0010
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 52s 821ms/step - accuracy: 0.6437 - loss: 0.6277 - val_accuracy: 0.6460 - val_loss: 0.6336 - learning_rate: 0.0010
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 52s 822ms/step - accuracy: 0.6463 - loss: 0.6398 - val_accuracy: 0.6440 - val_loss: 0.6251 - learning_rate: 0.0010
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 51s 817ms/step - accuracy: 0.6720 - loss: 0.

# 6.ĐÁNH GIÁ ĐỘ CHÍNH XÁC

In [7]:
# Dự đoán trên tập test
y_pred = model.predict(x_test_normalized)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Tính accuracy
accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f"Accuracy của mô hình sử dụng ResNet50: {accuracy}")




16/16 ━━━━━━━━━━━━━━━━━━━━ 12s 685ms/step
Accuracy của mô hình sử dụng ResNet50: 0.64
